In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings
import pandas_profiling as ppf

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error


test = pd.read_csv("dataset/test.csv")
train = pd.read_csv("dataset/train.csv")

### EDA

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [16]:
ppf.ProfileReport(train)

### Data Preprocessing
#### Missing Values and Drop Unnecessary Cols
`Age` has 19.9% missing value, use mean to fill in the missing value.


In [2]:
train['Age'] = train.Age.fillna(np.mean(train.Age))
test['Age'] = test.Age.fillna(np.mean(test.Age))
train['Fare'] = train.Fare.fillna(np.mean(train.Fare))
test['Fare'] = test.Fare.fillna(np.mean(test.Fare))
train.drop(['Name','Cabin','Ticket','Embarked'],axis = 1, inplace = True)
test.drop(['Name','Cabin','Ticket','Embarked'],axis = 1, inplace = True)

In [3]:
test.Fare.isnull().sum() # double check how many missing values

0

#### Encode `gender` variable
`gender` is categorical variable and we want to convert it into numerical data.

In [4]:
lab = LabelEncoder()
train.Sex = lab.fit_transform(train.Sex)
test.Sex = lab.fit_transform(test.Sex)

#### Normalize Some of the Variables
Nornalize `Age`, `Fare`  by MinMaxScaler

In [5]:
minmax = MinMaxScaler()
train.Age = minmax.fit_transform(np.array(train.Age).reshape(-1,1))
test.Age = minmax.fit_transform(np.array(test.Age).reshape(-1,1))
train.Fare = minmax.fit_transform(np.array(train.Fare).reshape(-1,1))
test.Fare = minmax.fit_transform(np.array(test.Fare).reshape(-1,1))

In [6]:
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,892,3,1,0.452723,0,0,0.015282
1,893,3,0,0.617566,1,0,0.013663
2,894,2,1,0.815377,0,0,0.018909
3,895,3,1,0.353818,0,0,0.016908
4,896,3,0,0.287881,1,1,0.023984


In [7]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,1,0.271174,1,0,0.014151
1,2,1,1,0,0.472229,1,0,0.139136
2,3,1,3,0,0.321438,0,0,0.015469
3,4,1,1,0,0.434531,1,0,0.103644
4,5,0,3,1,0.434531,0,0,0.015713


In [8]:
x = train.drop('Survived',axis = 1)
y = train.Survived

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3, random_state = 0)

In [10]:
x_train.shape, y_train.shape

((623, 7), (623,))

### Model
Use linear regression here.

In [11]:
lin = LinearRegression()
lin.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [12]:
y_pred = lin.predict(x_test)

In [13]:
mae = mean_absolute_error(y_pred,y_test)
mae

0.2931077454852208

In [14]:
prediction = lin.predict(test)
prediction

array([ 0.0760432 ,  0.4685533 ,  0.06589668,  0.12325543,  0.60638514,
        0.20474355,  0.6234152 ,  0.22995302,  0.69825173,  0.07044634,
        0.10243415,  0.33364852,  0.97182524,  0.01799975,  0.81300957,
        0.78186582,  0.2357001 ,  0.16007643,  0.59385596,  0.52939502,
        0.24286039,  0.21274993,  0.95362144,  0.48393795,  0.8284797 ,
       -0.0665964 ,  0.99727959,  0.15063594,  0.36666849,  0.01132595,
        0.09915861,  0.2156135 ,  0.52025904,  0.53849822,  0.39836769,
        0.17559205,  0.62169554,  0.67990965,  0.13588394,  0.12246205,
        0.02935139,  0.43200337,  0.03515153,  0.78641782,  0.82173626,
        0.13518444,  0.34121621,  0.10213842,  0.78617342,  0.54781276,
        0.43670956,  0.28660404,  0.7356821 ,  0.87749207,  0.26635408,
        0.02363697,  0.07258942,  0.13499382,  0.05718127,  1.01352453,
        0.18508738,  0.25464633,  0.17876273,  0.67310325,  0.49992918,
        0.78790109,  0.69815352,  0.33386196,  0.42811113,  0.73

In [15]:
submission = pd.DataFrame({'PassengerId':test.PassengerId, 'Survived':prediction})
submission.to_csv('submission_1.csv',index = False)